HW 1 - Question1

In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read the data
league_df = pd.read_csv('HW1_data.csv')

1. We have  684 observations

In [78]:
league_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   league    684 non-null    object 
 1   year      684 non-null    int64  
 2   position  684 non-null    int64  
 3   team      684 non-null    object 
 4   matches   684 non-null    int64  
 5   wins      684 non-null    int64  
 6   draws     684 non-null    int64  
 7   loses     684 non-null    int64  
 8   scored    684 non-null    int64  
 9   conceded  681 non-null    float64
 10  pts       684 non-null    int64  
dtypes: float64(1), int64(8), object(2)
memory usage: 58.9+ KB


2. Categorical fields:
    - League
    - Team

    Numerical fields:
    - Year
    - Position
    - Matches
    - Wins
    - Draws
    - Loses
    - Scored
    - Conceded
    - Pts

In [79]:
league_df.dtypes

league       object
year          int64
position      int64
team         object
matches       int64
wins          int64
draws         int64
loses         int64
scored        int64
conceded    float64
pts           int64
dtype: object

3.
- Number of League unique values: 6
- Number of Team unique values: 168

In [80]:
print("Number of league unique values:", league_df["league"].nunique(), "\n")

print("Number of Teams unique values:", league_df["team"].nunique())

Number of league unique values: 6 

Number of Teams unique values: 168


4.
The fields with null values are: 
- conceded:
    There are only 3 missing (null) values


In [81]:
league_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   league    684 non-null    object 
 1   year      684 non-null    int64  
 2   position  684 non-null    int64  
 3   team      684 non-null    object 
 4   matches   684 non-null    int64  
 5   wins      684 non-null    int64  
 6   draws     684 non-null    int64  
 7   loses     684 non-null    int64  
 8   scored    684 non-null    int64  
 9   conceded  681 non-null    float64
 10  pts       684 non-null    int64  
dtypes: float64(1), int64(8), object(2)
memory usage: 58.9+ KB


5. The number of teams in each league:
   - Bundesliga:    24
   - EPL:           30
   - La_liga:       30
   - Ligue_1:       29
   - RFPL:          25
   - Serie_A:       30

The minimum number of teams in a league is - 24 

In [82]:
teams_in_league = league_df.groupby("league")["team"].nunique()
print(teams_in_league, "\n")

print("The minimum number of teams in a league is", teams_in_league.min())

league
Bundesliga    24
EPL           30
La_liga       30
Ligue_1       29
RFPL          25
Serie_A       30
Name: team, dtype: int64 

The minimum number of teams in a league is 24


7. The league with the biggest gap between the higest median number of goal and the lowest one is Ligue_1 with the higest one in 2018 and lowest one in 2019

In [89]:
league_gb = league_df.groupby(['league', 'year'])['scored'].median()
print(league_gb)

league      year
Bundesliga  2014    44.0
            2015    44.0
            2016    44.5
            2017    43.5
            2018    53.0
            2019    48.0
EPL         2014    46.5
            2015    48.5
            2016    47.5
            2017    44.5
            2018    51.5
            2019    46.5
La_liga     2014    43.0
            2015    45.5
            2016    53.0
            2017    46.5
            2018    47.0
            2019    46.0
Ligue_1     2014    44.0
            2015    46.0
            2016    42.5
            2017    45.5
            2018    46.0
            2019    33.0
RFPL        2014    31.0
            2015    34.5
            2016    30.5
            2017    31.5
            2018    30.5
            2019    36.5
Serie_A     2014    49.0
            2015    46.5
            2016    56.0
            2017    51.0
            2018    51.5
            2019    51.5
Name: scored, dtype: float64


8. 
- There is only one mistake in the line 425 in the league Serie_A, for the team Crotone in 2017.
- In our opinion the mistake is from the pts column



In [134]:
def calculate_point(wins_col, draws_cols) :
    return (wins_col*3 + draws_cols*1)
    

league_df['calculate_point'] = league_df.apply(lambda x: calculate_point(x['wins'], x['draws']), axis = 1 )

comparaison = league_df['pts'] == league_df['calculate_point']

liste = []
for i in range(len(comparaison)) :
    if comparaison[i] == False : 
        liste.append(i)

league_df.iloc[liste]

        

,league,year,position,team,matches,wins,draws,loses,scored,conceded,pts,calculate_point,comparaison
425,Serie_A,2017,18,Crotone,38,9,8,21,40,66.0,180,35,False


9.
- if we are locking for the league with the most chees teams all years combined, Serie_A is the one. 
- if we want to know which league was with the most chees teams in each years then we have : 

             .2014 : Serie_A
             .2015 : EPL
             .2016 : Serie_A
             .2017 : Serie_A
             .2018 : Serie_A
             .2019 : Ligue_1

In [179]:
#create a new df without the line with null values
new_league_df = league_df.dropna()

#creat a new column that indicate if the team is a chess one or no.  
new_league_df['chees_team'] = new_league_df['scored'] < new_league_df['conceded']


nb_chees_team_per_yer = league_df.groupby(['year','league'])['chees_team'].sum()
print(nb_chees_team_per_yer)

league_gb = league_df.groupby('league')['chees_team'].sum()
print(league_gb, '\n')


league
Bundesliga    45
EPL           49
La_liga       45
Ligue_1       50
RFPL          44
Serie_A       55
Name: chees_team, dtype: int64 

year  league    
2014  Bundesliga     7
      EPL            8
      La_liga        8
      Ligue_1        8
      RFPL           8
      Serie_A       10
2015  Bundesliga     7
      EPL           10
      La_liga        7
      Ligue_1        8
      RFPL           7
      Serie_A        8
2016  Bundesliga     6
      EPL            7
      La_liga        8
      Ligue_1        6
      RFPL           7
      Serie_A        9
2017  Bundesliga     8
      EPL            6
      La_liga        7
      Ligue_1        8
      RFPL           7
      Serie_A        9
2018  Bundesliga     9
      EPL            9
      La_liga        6
      Ligue_1        9
      RFPL           9
      Serie_A       10
2019  Bundesliga     8
      EPL            9
      La_liga        9
      Ligue_1       11
      RFPL           6
      Serie_A        9
Name: chees_t

C:\Users\SAM\AppData\Local\Temp\ipykernel_278472\3672521040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_league_df['chees_team'] = new_league_df['scored'] < new_league_df['conceded']
